In [1]:
!git clone https://github.com/cs21s006/cs6910_assignment1

Cloning into 'cs6910_assignment1'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 96 (delta 35), reused 42 (delta 14), pack-reused 0
Unpacking objects: 100% (96/96), done.


In [2]:
cd cs6910_assignment1

/content/cs6910_assignment1


In [3]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 28.8 MB/s 
     |████████████████████████████████| 144 kB 49.3 MB/s 
     |████████████████████████████████| 181 kB 39.6 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=d199e8e12ad72a18e1faf07610bd0115ae49f88ab094b93d1d44b6140bc1a364
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [4]:
import wandb
import argparse
import numpy as np
from tqdm import tqdm
from keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split 

from neural_networks import NeuralNetwork, Optimizer, clip_gradients, losses
from utils import preprocess_data, make_batches, accuracy, gradient_sum

In [5]:
def mse_softmax_grad(y_true, y_pred):
    b, k = y_pred.shape
    y_minus_t = (y_pred - y_true)
    minus_y = np.repeat(y_pred, k, axis=1).reshape(b, k, k)
    batch_ones = np.tile(np.identity(k), b).T.reshape(b, k, k)
    prod = np.array([y_minus_t[i, :] @ (batch_ones-minus_y)[i, :] for i in range(b)])
    return np.multiply(y_pred, prod)


def train_and_evaluate(args):
    # Load Data
    (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
    X_train,X_val,y_train,y_val=train_test_split(X_train, y_train,test_size=0.1, random_state=1, stratify=y_train)
    (X_train, y_train) = preprocess_data(X_train, y_train)
    (X_test, y_test) = preprocess_data(X_test, y_test)
    (X_val,y_val) = preprocess_data(X_val,y_val)
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape,X_val.shape,y_val.shape)

    wandb.run.name = 'ep-'+str(args.epochs)+'-bs-'+str(args.batch_size)+'-lf-'+str(args.loss)+'-op-'+str(args.optimizer)+'-lr-'+ str(args.learning_rate) \
                      + '-nhl-'+str(args.num_layers)+'-shl-'+str(args.hidden_size)+ '-act-'+str(args.activation) \
                      +'-w_i-'+str(args.weight_init_method)+'-w_d-'+str(args.weight_decay)


    # Instantiate model
    architecture = [{'num_neurons': args.hidden_size, 'activation': args.activation, 'init_method': args.weight_init_method}
                    for _ in range(args.num_layers)]
    architecture.append({'num_neurons': 10, 'activation': 'softmax'})  # add output layer

    model = NeuralNetwork(input_dim=784, architecture=architecture)
    optimizer = Optimizer(model, algorithm=args.optimizer,
                          momentum=0.9, beta=0.9,
                          beta1=0.9, beta2=0.99, epsilon=1e-8,weight_decay=args.weight_decay)
    lr = args.learning_rate
    loss_fn, loss_grad_fn = losses[args.loss]

    if args.loss == 'mean_squared_error':
        loss_grad_fn = mse_softmax_grad

    # Train

    for epoch in range(args.epochs):
        running_loss, running_grad = .0, .0
        num_steps = (X_train.shape[0] // args.batch_size)
        progress_bar = tqdm(make_batches(X_train, y_train, args.batch_size),
                            total=num_steps)
        for (X_batch, y_batch) in progress_bar:
            # Forward
            y_pred = model.forward(X_batch)

            # Optimize
            gradients = optimizer.backpropagate(y_batch, y_pred, (loss_fn, loss_grad_fn))
            gradients = clip_gradients(gradients, clip_ratio=5.0, norm_type=2)
            optimizer.optimize(gradients, learning_rate=lr)

            # Track acc, loss and gradients
            loss = loss_fn(y_batch, y_pred)
            grad_sum = gradient_sum(gradients)
            acc = accuracy(y_batch, y_pred)
            progress_bar.set_description(
                f"epoch: {epoch}, lr: {lr:.5f} | loss: {loss:.4f}, acc(batch): {acc:.4f}, grad:{grad_sum:.4f}"
            )
            
            running_loss += loss
            running_grad += grad_sum
        
        # Evaluate train and test splits
        train_acc = accuracy(model.forward(X_train), y_train)
        y_val_pred = model.forward(X_val)
        val_acc = accuracy(y_val_pred,y_val)
        val_loss = loss_fn(y_val,y_val_pred)
        test_acc = accuracy(model.forward(X_test), y_test)
        print(f"acc(train): {train_acc:.4f}, acc(val): {val_acc:.4f}, acc(test): {test_acc:.4f}")
        print('_' * 99)
        
        # Log metrics to wandb.ai
        wandb.log({
            'train_acc': train_acc, 
            'val_acc': val_acc,
            'train_loss': running_loss/num_steps,
            'val_loss' : val_loss,
            'test_acc': test_acc,
            'epoch':epoch            
        })
    
    y_true = np.argmax(y_test, axis=1)
    y_pred = np.argmax(model.forward(X_test), axis=1)
    class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 
              'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankleboot']
    wandb.log({
      "confusion_matrix" : wandb.plot.confusion_matrix(probs=None,
                              y_true=y_true, preds=y_pred,
                              class_names=class_names)
    })

In [6]:
wandb.login()

sweep_config = {
    "name" : "cs6910_assignment1_fashion-mnist_mse_vs_xe_sweep",
    "description": "sweep comparing mean squared error and cross entropy loss functions",
    "method" : "grid",
    'metric': {
        'name': 'val_acc',
        'goal': 'maximize'
    },
    "parameters" : {
        "epochs" : {
            "values" : [20]
        },
        "batch_size": {
            "values": [128]
        },
        "loss": {
            "values": ['mean_squared_error', 'cross_entropy']
        },
        "optimizer": {
            "values": ["adam"]
        },
        "learning_rate" :{
            "values": [1e-4]
        },
        "num_layers": {
            "values": [4]
        },
        "hidden_size": {
            "values": [256]
        },
        "activation": {
            "values": ['tanh']
        },
        "weight_init_method":{
            "values":['He_normal']
        },
        "weight_decay":{
            "values":[0]
        }
    }
}
sweep_id = wandb.sweep(sweep_config, project="cs6910_assignment1", entity="cs21s006_cs21s043")
print('sweep_id: ', sweep_id)

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: cp83i9rw
Sweep URL: https://wandb.ai/cs21s006_cs21s043/cs6910_assignment1/sweeps/cp83i9rw
sweep_id:  cp83i9rw


In [7]:
def spawn_fn():
      with wandb.init(project="cs6910_assignment1", entity="cs21s006_cs21s043") as run:
        config = wandb.config
        train_and_evaluate(config)

In [8]:
wandb.agent(sweep_id, function=spawn_fn)

wandb: Agent Starting Run: nlskhadb with config:
wandb: 	activation: tanh
wandb: 	batch_size: 128
wandb: 	epochs: 20
wandb: 	hidden_size: 256
wandb: 	learning_rate: 0.0001
wandb: 	loss: mean_squared_error
wandb: 	num_layers: 4
wandb: 	optimizer: adam
wandb: 	weight_decay: 0
wandb: 	weight_init_method: He_normal
wandb: Currently logged in as: cs21s006_cs21s043 (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


4431872/4422102 [==============================] - 0s 0us/step
(54000, 784) (54000, 10) (10000, 784) (10000, 10) (6000, 784) (6000, 10)


epoch: 0, lr: 0.00010 | loss: 0.0189, acc(batch): 0.8839, grad:104041.2075: : 422it [00:44,  9.56it/s]                       


acc(train): 0.8486, acc(val): 0.8467, acc(test): 0.8372
___________________________________________________________________________________________________


epoch: 1, lr: 0.00010 | loss: 0.0181, acc(batch): 0.9018, grad:103704.1240: : 422it [00:43,  9.60it/s]


...

acc(train): 0.9216, acc(val): 0.8958, acc(test): 0.8796
___________________________________________________________________________________________________


epoch: 19, lr: 0.00010 | loss: 0.0227, acc(batch): 0.9464, grad:108164.5237: : 422it [00:40, 10.33it/s]                       


acc(train): 0.9230, acc(val): 0.8952, acc(test): 0.8803
___________________________________________________________________________________________________


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
test_acc,▁▃▄▄▅▆▆▆▇▇▇▇▇███████
train_acc,▁▂▃▄▅▅▅▆▆▆▆▇▇▇▇▇████
train_loss,█▅▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁
val_acc,▁▃▄▄▅▆▆▆▇▇▇▇████████
val_loss,█▆▅▅▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁
epoch,19
test_acc,0.8803
train_acc,0.92302
train_loss,0.02062
val_acc,0.89517


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [ ]:
# wandb.agent("1463mzwx", function=spawn_fn, count=10)

wandb: Agent Starting Run: 7uitvlls with config:
wandb: 	activation: tanh
wandb: 	batch_size: 64
wandb: 	epochs: 20
wandb: 	hidden_size: 16
wandb: 	learning_rate: 0.0001
wandb: 	loss: cross_entropy
wandb: 	num_layers: 5
wandb: 	optimizer: adam
wandb: 	weight_decay: 0
wandb: 	weight_init_method: Xavier_normal
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


(54000, 784) (54000, 10) (10000, 784) (10000, 10) (6000, 784) (6000, 10)


epoch: 0, lr: 0.00010 | loss: 0.0817, acc(batch): 0.8958, grad:13974.4253: : 844it [00:10, 77.94it/s]                       


acc(train): 0.7300, acc(val): 0.7305, acc(test): 0.7233
___________________________________________________________________________________________________


epoch: 1, lr: 0.00010 | loss: 0.0503, acc(batch): 0.9375, grad:15169.5247: : 844it [00:10, 77.85it/s]


...

epoch: 18, lr: 0.00050 | loss: 0.0388, acc(batch): 0.8125, grad:18722.3485: : 1688it [00:25, 66.89it/s]


acc(train): 0.8592, acc(val): 0.8552, acc(test): 0.8421
___________________________________________________________________________________________________


epoch: 19, lr: 0.00050 | loss: 0.0389, acc(batch): 0.8125, grad:18683.6124: : 1688it [00:24, 67.53it/s]                        


acc(train): 0.8603, acc(val): 0.8558, acc(test): 0.8423
___________________________________________________________________________________________________


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
test_acc,▁▃▅▆▆▆▇▇▇▇▇▇▇███████
train_acc,▁▃▅▅▆▆▆▇▇▇▇▇▇▇██████
train_loss,█▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▃▅▆▆▆▇▇▇▇▇▇████████
val_loss,█▅▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
epoch,19
test_acc,0.8423
train_acc,0.86026
train_loss,0.03931
val_acc,0.85583
